In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

# Load dataset
df = pd.read_csv("powerconsumption.csv")

# Preprocessing steps
df['Temperature'] = df['Temperature'].fillna(df['Temperature'].mean())
df['Humidity'].fillna(df['Humidity'].mean(), inplace=True)
df['DiffuseFlows'].fillna(df['DiffuseFlows'].mean(), inplace=True)

df["Month"] = pd.to_datetime(df["Datetime"]).dt.month
df["Year"] = pd.to_datetime(df["Datetime"]).dt.year
df["Date"] = pd.to_datetime(df["Datetime"]).dt.date
df["Time"] = pd.to_datetime(df["Datetime"]).dt.time
df["Week"] = pd.to_datetime(df["Datetime"]).dt.isocalendar().week
df["Day"] = pd.to_datetime(df["Datetime"]).dt.day_name()
df = df.set_index("Datetime")
df.index = pd.to_datetime(df.index)

# Perform one-hot encoding on the 'Day' column
df = pd.get_dummies(df, columns=['Day'])

# Define features and target variable
X = df[['Temperature', 'Humidity', 'WindSpeed', 'GeneralDiffuseFlows', 'DiffuseFlows','Month',
        'Day_Friday', 'Day_Monday', 'Day_Saturday', 'Day_Sunday', 'Day_Thursday', 'Day_Tuesday', 'Day_Wednesday']]
y = df['PowerConsumption']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize base models
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)
rf_model = RandomForestRegressor(random_state=42)

# Train base models
xgb_model.fit(X_train_scaled, y_train)
rf_model.fit(X_train_scaled, y_train)

# Make predictions on the test set
xgb_pred = xgb_model.predict(X_test_scaled)
rf_pred = rf_model.predict(X_test_scaled)

# Stack predictions
stacked_pred = np.column_stack((xgb_pred, rf_pred))

# Initialize meta-model (XGBoost)
meta_model = XGBRegressor(objective='reg:squarederror', random_state=42)

# Train meta-model on stacked predictions
meta_model.fit(stacked_pred, y_test)

# Ensure models and scaler are defined and trained
assert xgb_model is not None
assert rf_model is not None
assert meta_model is not None
assert scaler is not None


C:\Users\91798\AppData\Local\Temp\ipykernel_16424\627073023.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Temperature'].fillna(df['Temperature'].mean(), inplace=True)
C:\Users\91798\AppData\Local\Temp\ipykernel_16424\627073023.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as 

In [2]:
# !pip install xgboost


   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.8 MB 259.2 kB/s eta 0:06:25
   ---------------------------------------- 0.1/99.8 MB 508.4 kB/s eta 0:03:17
   ---------------------------------------- 0.2/99.8 MB 1.1 MB/s eta 0:01:33
   ---------------------------------------- 0.5/99.8 MB 2.2 MB/s eta 0:00:45
   ---------------------------------------- 0.9/99.8 MB 3.2 MB/s eta 0:00:31
   ---------------------------------------- 1.1/99.8 MB 3.4 MB/s eta 0:00:30
    --------------------------------------- 1.3/99.8 MB 3.5 MB/s eta 0:00:29
    --------------------------------------- 1.4/99.8 MB 3.6 MB/s eta 0:00:28
    --------------------------------------- 1.4/99.8 MB 3.6 MB/s eta 0:00:28
    --------------------------------------- 1.4/99.8 MB 3.6 MB/s eta 0:00:28
    ------------

In [4]:
from scipy.optimize import minimize

def optimize_power_consumption(current_conditions, xgb_model, rf_model, meta_model, scaler):
    """
    Optimizes the power consumption based on current conditions.
    
    Parameters:
    - current_conditions: A dictionary containing the current values of features.
    - xgb_model: The trained XGBoost model.
    - rf_model: The trained Random Forest model.
    - meta_model: The trained meta-model for stacking.
    - scaler: The StandardScaler used to standardize the features.
    
    Returns:
    - optimized_conditions: A dictionary containing the optimized feature values.
    """
    
    # Extract feature values from the dictionary
    initial_features = np.array([
        current_conditions['Temperature'],
        current_conditions['Humidity'],
        current_conditions['WindSpeed'],
        current_conditions['GeneralDiffuseFlows'],
        current_conditions['DiffuseFlows'],
        current_conditions['Month'],
        current_conditions['Day_Friday'],
        current_conditions['Day_Monday'],
        current_conditions['Day_Saturday'],
        current_conditions['Day_Sunday'],
        current_conditions['Day_Thursday'],
        current_conditions['Day_Tuesday'],
        current_conditions['Day_Wednesday']
    ]).reshape(1, -1)
    
    # Standardize the initial feature values
    initial_features_scaled = scaler.transform(initial_features)
    
    # Objective function to minimize
    def objective(features_scaled):
        features = features_scaled.reshape(1, -1)
        
        # Predict using the base models
        xgb_pred = xgb_model.predict(features)
        rf_pred = rf_model.predict(features)
        
        # Stack the predictions
        stacked_pred = np.column_stack((xgb_pred, rf_pred))
        
        # Predict using the meta-model
        meta_pred = meta_model.predict(stacked_pred)
        
        # Return the predicted power consumption
        return meta_pred[0]
    
    # Bounds for the optimization (you can adjust these as necessary)
    bounds = [
        (initial_features_scaled[0, 0] - 1, initial_features_scaled[0, 0] + 1),  # Temperature
        (initial_features_scaled[0, 1] - 10, initial_features_scaled[0, 1] + 10),  # Humidity
        (initial_features_scaled[0, 2] - 0.1, initial_features_scaled[0, 2] + 0.1),  # WindSpeed
        (initial_features_scaled[0, 3] - 0.1, initial_features_scaled[0, 3] + 0.1),  # GeneralDiffuseFlows
        (initial_features_scaled[0, 4] - 0.1, initial_features_scaled[0, 4] + 0.1),  # DiffuseFlows
        (initial_features_scaled[0, 5], initial_features_scaled[0, 5]),  # Month (fixed)
        (0, 1),  # Day_Friday
        (0, 1),  # Day_Monday
        (0, 1),  # Day_Saturday
        (0, 1),  # Day_Sunday
        (0, 1),  # Day_Thursday
        (0, 1),  # Day_Tuesday
        (0, 1)   # Day_Wednesday
    ]
    
    # Run the optimization
    result = minimize(objective, initial_features_scaled, bounds=bounds, method='L-BFGS-B')
    
    # Get the optimized feature values
    optimized_features_scaled = result.x
    
    # Inverse transform the optimized features to get them back to original scale
    optimized_features = scaler.inverse_transform(optimized_features_scaled.reshape(1, -1))
    
    # Create a dictionary of optimized conditions
    optimized_conditions = {
        'Temperature': optimized_features[0, 0],
        'Humidity': optimized_features[0, 1],
        'WindSpeed': optimized_features[0, 2],
        'GeneralDiffuseFlows': optimized_features[0, 3],
        'DiffuseFlows': optimized_features[0, 4],
        'Month': optimized_features[0, 5],
        'Day_Friday': int(optimized_features[0, 6]),
        'Day_Monday': int(optimized_features[0, 7]),
        'Day_Saturday': int(optimized_features[0, 8]),
        'Day_Sunday': int(optimized_features[0, 9]),
        'Day_Thursday': int(optimized_features[0, 10]),
        'Day_Tuesday': int(optimized_features[0, 11]),
        'Day_Wednesday': int(optimized_features[0, 12])
    }
    
    return optimized_conditions

# Example usage
current_conditions = {
    'Temperature': 20,
    'Humidity': 50,
    'WindSpeed': 0.1,
    'GeneralDiffuseFlows': 0.1,
    'DiffuseFlows': 0.1,
    'Month': 5,
    'Day_Friday': 0,
    'Day_Monday': 1,
    'Day_Saturday': 0,
    'Day_Sunday': 0,
    'Day_Thursday': 0,
    'Day_Tuesday': 0,
    'Day_Wednesday': 0
}

# Make sure xgb_model, rf_model, meta_model, and scaler are defined and trained before calling this function
optimized_conditions = optimize_power_consumption(current_conditions, xgb_model, rf_model, meta_model, scaler)
print("Optimized Conditions:", optimized_conditions)


c:\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


ValueError: 'x0' must only have one dimension.

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from scipy.optimize import minimize

# Load dataset
df = pd.read_csv("powerconsumption.csv")

# Preprocessing steps
df['Temperature'].fillna(df['Temperature'].mean(), inplace=True)
df['Humidity'].fillna(df['Humidity'].mean(), inplace=True)
df['DiffuseFlows'].fillna(df['DiffuseFlows'].mean(), inplace=True)

df["Month"] = pd.to_datetime(df["Datetime"]).dt.month
df["Year"] = pd.to_datetime(df["Datetime"]).dt.year
df["Date"] = pd.to_datetime(df["Datetime"]).dt.date
df["Time"] = pd.to_datetime(df["Datetime"]).dt.time
df["Week"] = pd.to_datetime(df["Datetime"]).dt.isocalendar().week
df["Day"] = pd.to_datetime(df["Datetime"]).dt.day_name()
df = df.set_index("Datetime")
df.index = pd.to_datetime(df.index)

# Perform one-hot encoding on the 'Day' column
df = pd.get_dummies(df, columns=['Day'])

# Define features and target variable
X = df[['Temperature', 'Humidity', 'WindSpeed', 'GeneralDiffuseFlows', 'DiffuseFlows','Month',
        'Day_Friday', 'Day_Monday', 'Day_Saturday', 'Day_Sunday', 'Day_Thursday', 'Day_Tuesday', 'Day_Wednesday']]
y = df['PowerConsumption']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize base models
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)
rf_model = RandomForestRegressor(random_state=42)

# Train base models
xgb_model.fit(X_train_scaled, y_train)
rf_model.fit(X_train_scaled, y_train)

# Make predictions on the test set
xgb_pred = xgb_model.predict(X_test_scaled)
rf_pred = rf_model.predict(X_test_scaled)

# Stack predictions
stacked_pred = np.column_stack((xgb_pred, rf_pred))

# Initialize meta-model (XGBoost)
meta_model = XGBRegressor(objective='reg:squarederror', random_state=42)

# Train meta-model on stacked predictions
meta_model.fit(stacked_pred, y_test)

# Ensure models and scaler are defined and trained
assert xgb_model is not None
assert rf_model is not None
assert meta_model is not None
assert scaler is not None

def optimize_power_consumption(current_conditions, xgb_model, rf_model, meta_model, scaler):
    """
    Optimizes the power consumption based on current conditions.
    
    Parameters:
    - current_conditions: A dictionary containing the current values of features.
    - xgb_model: The trained XGBoost model.
    - rf_model: The trained Random Forest model.
    - meta_model: The trained meta-model for stacking.
    - scaler: The StandardScaler used to standardize the features.
    
    Returns:
    - optimized_conditions: A dictionary containing the optimized feature values.
    """
    
    # Extract feature values from the dictionary
    initial_features = np.array([
        current_conditions['Temperature'],
        current_conditions['Humidity'],
        current_conditions['WindSpeed'],
        current_conditions['GeneralDiffuseFlows'],
        current_conditions['DiffuseFlows'],
        current_conditions['Month'],
        current_conditions['Day_Friday'],
        current_conditions['Day_Monday'],
        current_conditions['Day_Saturday'],
        current_conditions['Day_Sunday'],
        current_conditions['Day_Thursday'],
        current_conditions['Day_Tuesday'],
        current_conditions['Day_Wednesday']
    ])
    
    # Standardize the initial feature values
    initial_features_scaled = scaler.transform(initial_features.reshape(1, -1))
    
    # Objective function to minimize
    def objective(features_scaled):
        features = features_scaled.reshape(1, -1)
        
        # Predict using the base models
        xgb_pred = xgb_model.predict(features)
        rf_pred = rf_model.predict(features)
        
        # Stack the predictions
        stacked_pred = np.column_stack((xgb_pred, rf_pred))
        
        # Predict using the meta-model
        meta_pred = meta_model.predict(stacked_pred)
        
        # Return the predicted power consumption
        return meta_pred[0]
    
    # Bounds for the optimization (you can adjust these as necessary)
    bounds = [
        (initial_features_scaled[0, 0] - 1, initial_features_scaled[0, 0] + 1),  # Temperature
        (initial_features_scaled[0, 1] - 10, initial_features_scaled[0, 1] + 10),  # Humidity
        (initial_features_scaled[0, 2] - 0.1, initial_features_scaled[0, 2] + 0.1),  # WindSpeed
        (initial_features_scaled[0, 3] - 0.1, initial_features_scaled[0, 3] + 0.1),  # GeneralDiffuseFlows
        (initial_features_scaled[0, 4] - 0.1, initial_features_scaled[0, 4] + 0.1),  # DiffuseFlows
        (initial_features_scaled[0, 5], initial_features_scaled[0, 5]),  # Month (fixed)
        (0, 1),  # Day_Friday
        (0, 1),  # Day_Monday
        (0, 1),  # Day_Saturday
        (0, 1),  # Day_Sunday
        (0, 1),  # Day_Thursday
        (0, 1),  # Day_Tuesday
        (0, 1)   # Day_Wednesday
    ]
    
    # Run the optimization
    result = minimize(objective, initial_features_scaled.flatten(), bounds=bounds, method='L-BFGS-B')
    
    # Get the optimized feature values
    optimized_features_scaled = result.x
    
    # Inverse transform the optimized features to get them back to original scale
    optimized_features = scaler.inverse_transform(optimized_features_scaled.reshape(1, -1))
    
    # Create a dictionary of optimized conditions
    optimized_conditions = {
        'Temperature': optimized_features[0, 0],
        'Humidity': optimized_features[0, 1],
        'WindSpeed': optimized_features[0, 2],
        'GeneralDiffuseFlows': optimized_features[0, 3],
        'DiffuseFlows': optimized_features[0, 4],
        'Month': optimized_features[0, 5],
        'Day_Friday': int(round(optimized_features[0, 6])),
        'Day_Monday': int(round(optimized_features[0, 7])),
        'Day_Saturday': int(round(optimized_features[0, 8])),
        'Day_Sunday': int(round(optimized_features[0, 9])),
        'Day_Thursday': int(round(optimized_features[0, 10])),
        'Day_Tuesday': int(round(optimized_features[0, 11])),
        'Day_Wednesday': int(round(optimized_features[0, 12]))
    }
    
    return optimized_conditions

# Example usage
current_conditions = {
    'Temperature': 20,
    'Humidity': 50,
    'WindSpeed': 0.1,
    'GeneralDiffuseFlows': 0.1,
    'DiffuseFlows': 0.1,
    'Month': 5,
    'Day_Friday': 0,
    'Day_Monday': 1,
    'Day_Saturday': 0,
    'Day_Sunday': 0,
    'Day_Thursday': 0,
    'Day_Tuesday': 0,
    'Day_Wednesday': 0
}

# Make sure xgb_model, rf_model, meta_model, and scaler are defined and trained before calling this function
optimized_conditions = optimize_power_consumption(current_conditions, xgb_model, rf_model, meta_model, scaler)
print("Optimized Conditions:", optimized_conditions)


C:\Users\91798\AppData\Local\Temp\ipykernel_16424\2027234100.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Temperature'].fillna(df['Temperature'].mean(), inplace=True)
C:\Users\91798\AppData\Local\Temp\ipykernel_16424\2027234100.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves a

Optimized Conditions: {'Temperature': 20.0, 'Humidity': 50.0, 'WindSpeed': 0.10000000000000009, 'GeneralDiffuseFlows': 0.09999999999999432, 'DiffuseFlows': 0.09999999999999432, 'Month': 5.0, 'Day_Friday': 0, 'Day_Monday': 0, 'Day_Saturday': 0, 'Day_Sunday': 0, 'Day_Thursday': 0, 'Day_Tuesday': 0, 'Day_Wednesday': 0}


c:\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from scipy.optimize import minimize

# Load dataset
df = pd.read_csv("powerconsumption.csv")

# Preprocessing steps
df['Temperature'] = df['Temperature'].fillna(df['Temperature'].mean())
df['Humidity'] = df['Humidity'].fillna(df['Humidity'].mean())
df['DiffuseFlows'] = df['DiffuseFlows'].fillna(df['DiffuseFlows'].mean())

df["Month"] = pd.to_datetime(df["Datetime"]).dt.month
df["Year"] = pd.to_datetime(df["Datetime"]).dt.year
df["Date"] = pd.to_datetime(df["Datetime"]).dt.date
df["Time"] = pd.to_datetime(df["Datetime"]).dt.time
df["Week"] = pd.to_datetime(df["Datetime"]).dt.isocalendar().week
df["Day"] = pd.to_datetime(df["Datetime"]).dt.day_name()
df = df.set_index("Datetime")
df.index = pd.to_datetime(df.index)

# Perform one-hot encoding on the 'Day' column
df = pd.get_dummies(df, columns=['Day'])

# Define features and target variable
X = df[['Temperature', 'Humidity', 'WindSpeed', 'GeneralDiffuseFlows', 'DiffuseFlows', 'Month',
        'Day_Friday', 'Day_Monday', 'Day_Saturday', 'Day_Sunday', 'Day_Thursday', 'Day_Tuesday', 'Day_Wednesday']]
y = df['PowerConsumption']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize base models
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)
rf_model = RandomForestRegressor(random_state=42)

# Train base models
xgb_model.fit(X_train_scaled, y_train)
rf_model.fit(X_train_scaled, y_train)

# Make predictions on the test set
xgb_pred = xgb_model.predict(X_test_scaled)
rf_pred = rf_model.predict(X_test_scaled)

# Stack predictions
stacked_pred = np.column_stack((xgb_pred, rf_pred))

# Initialize meta-model (XGBoost)
meta_model = XGBRegressor(objective='reg:squarederror', random_state=42)

# Train meta-model on stacked predictions
meta_model.fit(stacked_pred, y_test)

# Ensure models and scaler are defined and trained
assert xgb_model is not None
assert rf_model is not None
assert meta_model is not None
assert scaler is not None

def optimize_power_consumption(current_conditions, xgb_model, rf_model, meta_model, scaler):
    """
    Optimizes the power consumption based on current conditions.
    
    Parameters:
    - current_conditions: A dictionary containing the current values of features.
    - xgb_model: The trained XGBoost model.
    - rf_model: The trained Random Forest model.
    - meta_model: The trained meta-model for stacking.
    - scaler: The StandardScaler used to standardize the features.
    
    Returns:
    - optimized_conditions: A dictionary containing the optimized feature values.
    """
    
    # Extract feature values from the dictionary
    initial_features = pd.DataFrame([current_conditions])
    
    # Standardize the initial feature values
    initial_features_scaled = scaler.transform(initial_features)
    
    # Objective function to minimize
    def objective(features_scaled):
        features = features_scaled.reshape(1, -1)
        
        # Predict using the base models
        xgb_pred = xgb_model.predict(features)
        rf_pred = rf_model.predict(features)
        
        # Stack the predictions
        stacked_pred = np.column_stack((xgb_pred, rf_pred))
        
        # Predict using the meta-model
        meta_pred = meta_model.predict(stacked_pred)
        
        # Return the predicted power consumption
        return meta_pred[0]
    
    # Bounds for the optimization (you can adjust these as necessary)
    bounds = [
        (initial_features_scaled[0, 0] - 1, initial_features_scaled[0, 0] + 1),  # Temperature
        (initial_features_scaled[0, 1] - 10, initial_features_scaled[0, 1] + 10),  # Humidity
        (initial_features_scaled[0, 2] - 0.1, initial_features_scaled[0, 2] + 0.1),  # WindSpeed
        (initial_features_scaled[0, 3] - 0.1, initial_features_scaled[0, 3] + 0.1),  # GeneralDiffuseFlows
        (initial_features_scaled[0, 4] - 0.1, initial_features_scaled[0, 4] + 0.1),  # DiffuseFlows
        (initial_features_scaled[0, 5], initial_features_scaled[0, 5]),  # Month (fixed)
        (0, 1),  # Day_Friday
        (0, 1),  # Day_Monday
        (0, 1),  # Day_Saturday
        (0, 1),  # Day_Sunday
        (0, 1),  # Day_Thursday
        (0, 1),  # Day_Tuesday
        (0, 1)   # Day_Wednesday
    ]
    
    # Run the optimization
    result = minimize(objective, initial_features_scaled.flatten(), bounds=bounds, method='L-BFGS-B')
    
    # Get the optimized feature values
    optimized_features_scaled = result.x
    
    # Inverse transform the optimized features to get them back to original scale
    optimized_features = scaler.inverse_transform(optimized_features_scaled.reshape(1, -1))
    
    # Create a dictionary of optimized conditions
    optimized_conditions = {
        'Temperature': optimized_features[0, 0],
        'Humidity': optimized_features[0, 1],
        'WindSpeed': optimized_features[0, 2],
        'GeneralDiffuseFlows': optimized_features[0, 3],
        'DiffuseFlows': optimized_features[0, 4],
        'Month': optimized_features[0, 5],
        'Day_Friday': int(round(optimized_features[0, 6])),
        'Day_Monday': int(round(optimized_features[0, 7])),
        'Day_Saturday': int(round(optimized_features[0, 8])),
        'Day_Sunday': int(round(optimized_features[0, 9])),
        'Day_Thursday': int(round(optimized_features[0, 10])),
        'Day_Tuesday': int(round(optimized_features[0, 11])),
        'Day_Wednesday': int(round(optimized_features[0, 12]))
    }
    
    return optimized_conditions

# Example usage
current_conditions = {
    'Temperature': 20,
    'Humidity': 50,
    'WindSpeed': 0.1,
    'GeneralDiffuseFlows': 0.1,
    'DiffuseFlows': 0.1,
    'Month': 5,
    'Day_Friday': 0,
    'Day_Monday': 1,
    'Day_Saturday': 0,
    'Day_Sunday': 0,
    'Day_Thursday': 0,
    'Day_Tuesday': 0,
    'Day_Wednesday': 0
}

# Make sure xgb_model, rf_model, meta_model, and scaler are defined and trained before calling this function
optimized_conditions = optimize_power_consumption(current_conditions, xgb_model, rf_model, meta_model, scaler)
print("Optimized Conditions:", optimized_conditions)


Optimized Conditions: {'Temperature': 20.0, 'Humidity': 50.0, 'WindSpeed': 0.10000000000000009, 'GeneralDiffuseFlows': 0.09999999999999432, 'DiffuseFlows': 0.09999999999999432, 'Month': 5.0, 'Day_Friday': 0, 'Day_Monday': 0, 'Day_Saturday': 0, 'Day_Sunday': 0, 'Day_Thursday': 0, 'Day_Tuesday': 0, 'Day_Wednesday': 0}


In [7]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
import matplotlib.pyplot as plt

# Load dataset
df = pd.read_csv("powerconsumption.csv")

# Preprocessing steps
df['Temperature'] = df['Temperature'].fillna(df['Temperature'].mean())
df['Humidity'] = df['Humidity'].fillna(df['Humidity'].mean())
df['DiffuseFlows'] = df['DiffuseFlows'].fillna(df['DiffuseFlows'].mean())

df["Month"] = pd.to_datetime(df["Datetime"]).dt.month
df["Year"] = pd.to_datetime(df["Datetime"]).dt.year
df["Date"] = pd.to_datetime(df["Datetime"]).dt.date
df["Time"] = pd.to_datetime(df["Datetime"]).dt.time
df["Week"] = pd.to_datetime(df["Datetime"]).dt.isocalendar().week
df["Day"] = pd.to_datetime(df["Datetime"]).dt.day_name()
df = df.set_index("Datetime")
df.index = pd.to_datetime(df.index)

# Perform one-hot encoding on the 'Day' column
df = pd.get_dummies(df, columns=['Day'])

# Define features and target variable
X = df[['Temperature', 'Humidity', 'WindSpeed', 'GeneralDiffuseFlows', 'DiffuseFlows','Month',
        'Day_Friday', 'Day_Monday', 'Day_Saturday', 'Day_Sunday', 'Day_Thursday', 'Day_Tuesday', 'Day_Wednesday']]
y = df['PowerConsumption']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize base models
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)
rf_model = RandomForestRegressor(random_state=42)

# Train base models
xgb_model.fit(X_train_scaled, y_train)
rf_model.fit(X_train_scaled, y_train)

# Make predictions on the test set
xgb_pred = xgb_model.predict(X_test_scaled)
rf_pred = rf_model.predict(X_test_scaled)

# Stack predictions
stacked_pred = np.column_stack((xgb_pred, rf_pred))

# Initialize meta-model (XGBoost)
meta_model = XGBRegressor(objective='reg:squarederror', random_state=42)

# Train meta-model on stacked predictions
meta_model.fit(stacked_pred, y_test)

# Function to predict power consumption for a given date
def predict_power_consumption(input_date):
    # Process input date
    input_date = pd.to_datetime(input_date)
    month = input_date.month
    day_of_week = input_date.day_name()
    
    # Extract features for the input date
    input_features = [df['Temperature'].mean(), df['Humidity'].mean(), df['DiffuseFlows'].mean(),
                      month, 0, 0, 0, 0, 0, 0, 0, 0, 0]  # Assuming average weather conditions
    
    # Adjust day of the week in input features
    input_features[df.columns.get_loc('Day_' + day_of_week)] = 1
    
    # Standardize input features
    input_features_scaled = scaler.transform([input_features])
    
    # Make predictions using base models
    xgb_pred = xgb_model.predict(input_features_scaled)
    rf_pred = rf_model.predict(input_features_scaled)
    
    # Stack predictions
    stacked_pred = np.column_stack((xgb_pred, rf_pred))
    
    # Make predictions using meta-model
    meta_pred = meta_model.predict(stacked_pred)
    
    return meta_pred[0]

# Get input date from user
input_date = input("Enter the date (YYYY-MM-DD): ")

# Predict power consumption for the input date
predicted_power_consumption = predict_power_consumption(input_date)
print("Predicted Power Consumption on {}: {:.2f}".format(input_date, predicted_power_consumption))

# Optionally, you can plot the actual vs. predicted values for visualization
# Note: You may need to adjust the plotting code depending on your specific needs
# For example:
# plt.scatter(y_test, meta_pred_test)
# plt.xlabel('Actual Power Consumption')
# plt.ylabel('Predicted Power Consumption')
# plt.title('Actual vs. Predicted Power Consumption')
# plt.show()


IndexError: list assignment index out of range

In [10]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
import matplotlib.pyplot as plt

# Load dataset
df = pd.read_csv("powerconsumption.csv")

# Preprocessing steps
df['Temperature'] = df['Temperature'].fillna(df['Temperature'].mean())
df['Humidity'] = df['Humidity'].fillna(df['Humidity'].mean())
df['DiffuseFlows'] = df['DiffuseFlows'].fillna(df['DiffuseFlows'].mean())

df["Month"] = pd.to_datetime(df["Datetime"]).dt.month
df["Year"] = pd.to_datetime(df["Datetime"]).dt.year
df["Date"] = pd.to_datetime(df["Datetime"]).dt.date
df["Time"] = pd.to_datetime(df["Datetime"]).dt.time
df["Week"] = pd.to_datetime(df["Datetime"]).dt.isocalendar().week
df["Day"] = pd.to_datetime(df["Datetime"]).dt.day_name()
df = df.set_index("Datetime")
df.index = pd.to_datetime(df.index)

# Perform one-hot encoding on the 'Day' column
df = pd.get_dummies(df, columns=['Day'])

# Define features and target variable
X = df[['Temperature', 'Humidity', 'WindSpeed', 'GeneralDiffuseFlows', 'DiffuseFlows','Month',
        'Day_Friday', 'Day_Monday', 'Day_Saturday', 'Day_Sunday', 'Day_Thursday', 'Day_Tuesday', 'Day_Wednesday']]
y = df['PowerConsumption']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize base models
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)
rf_model = RandomForestRegressor(random_state=42)

# Train base models
xgb_model.fit(X_train_scaled, y_train)
rf_model.fit(X_train_scaled, y_train)

# Make predictions on the test set
xgb_pred = xgb_model.predict(X_test_scaled)
rf_pred = rf_model.predict(X_test_scaled)

# Stack predictions
stacked_pred = np.column_stack((xgb_pred, rf_pred))

# Initialize meta-model (XGBoost)
meta_model = XGBRegressor(objective='reg:squarederror', random_state=42)

# Train meta-model on stacked predictions
meta_model.fit(stacked_pred, y_test)

# Function to predict power consumption for a given date
def predict_power_consumption(input_date):
    # Process input date
    input_date = pd.to_datetime(input_date)
    month = input_date.month
    day_of_week = input_date.day_name()
    
    # Extract features for the input date
    input_features = [df['Temperature'].mean(), df['Humidity'].mean(), 0, df['DiffuseFlows'].mean(),
                      month, 0, 0, 0, 0, 0, 0, 0, 0]  # Assuming average weather conditions
    
    # Adjust day of the week in input features if the column exists
    day_column = 'Day_' + day_of_week
    if day_column in df.columns:
        input_features[df.columns.get_loc(day_column)] = 1
    
    # Standardize input features
    input_features_scaled = scaler.transform([input_features])
    
    # Make predictions using base models
    xgb_pred = xgb_model.predict(input_features_scaled)
    rf_pred = rf_model.predict(input_features_scaled)
    
    # Stack predictions
    stacked_pred = np.column_stack((xgb_pred, rf_pred))
    
    # Make predictions using meta-model
    meta_pred = meta_model.predict(stacked_pred)
    
    return meta_pred[0]

# Get input date from user
input_date = input("Enter the date (YYYY-MM-DD): ")

# Predict power consumption for the input date
predicted_power_consumption = predict_power_consumption(input_date)
print("Predicted Power Consumption on {}: {:.2f}".format(input_date, predicted_power_consumption))




IndexError: list assignment index out of range

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from statsmodels.tsa.api import VAR
from datetime import datetime

# Load the dataset
df = pd.read_csv("powerconsumption.csv")

# Fill missing values with mean
df['Temperature'].fillna(df['Temperature'].mean(), inplace=True)
df['Humidity'].fillna(df['Humidity'].mean(), inplace=True)
df['DiffuseFlows'].fillna(df['DiffuseFlows'].mean(), inplace=True)

# Convert datetime column to datetime format and extract additional features
df["Datetime"] = pd.to_datetime(df["Datetime"])
df["Month"] = df["Datetime"].dt.month
df["Year"] = df["Datetime"].dt.year
df["Date"] = df["Datetime"].dt.date
df["Time"] = df["Datetime"].dt.time
df["Week"] = df["Datetime"].dt.isocalendar().week
df["Day"] = df["Datetime"].dt.day_name()

# Perform one-hot encoding on the 'Day' column
df = pd.get_dummies(df, columns=['Day'])

# Define features and target variable
X = df[['Temperature', 'Humidity', 'WindSpeed', 'GeneralDiffuseFlows', 'DiffuseFlows','Month',
        'Day_Friday', 'Day_Monday', 'Day_Saturday', 'Day_Sunday', 'Day_Thursday', 'Day_Tuesday', 'Day_Wednesday']]
y = df['PowerConsumption']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize base models
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)
rf_model = RandomForestRegressor(random_state=42)

# Train base models
xgb_model.fit(X_train_scaled, y_train)
rf_model.fit(X_train_scaled, y_train)

# Make predictions on the test set
xgb_pred = xgb_model.predict(X_test_scaled)
rf_pred = rf_model.predict(X_test_scaled)

# Stack predictions
stacked_pred = np.column_stack((xgb_pred, rf_pred))

# Initialize meta-model (XGBoost)
meta_model = XGBRegressor(objective='reg:squarederror', random_state=42)

# Train meta-model on stacked predictions
meta_model.fit(stacked_pred, y_test)

# Fit VAR model
var_model = VAR(df[['Temperature', 'Humidity', 'WindSpeed', 'GeneralDiffuseFlows', 'DiffuseFlows']])
var_model_fitted = var_model.fit()

# Function to predict future values using VAR model
def predict_var(var_model_fitted, steps):
    lag_order = var_model_fitted.k_ar
    var_pred = var_model_fitted.forecast(var_data.values[-lag_order:], steps=steps)
    return var_pred

# Function to create static features for future dates
def create_static_features(future_dates):
    static_features_list = []
    for date in future_dates:
        month = date.month
        day_of_week = date.dayofweek
        static_features = [
            month,
            1 if day_of_week == 4 else 0,  # Friday
            1 if day_of_week == 0 else 0,  # Monday
            1 if day_of_week == 5 else 0,  # Saturday
            1 if day_of_week == 6 else 0,  # Sunday
            1 if day_of_week == 3 else 0,  # Thursday
            1 if day_of_week == 1 else 0,  # Tuesday
            1 if day_of_week == 2 else 0,  # Wednesday
        ]
        static_features_list.append(static_features)
    return pd.DataFrame(static_features_list, index=future_dates, columns=[
        'Month', 'Day_Friday', 'Day_Monday', 'Day_Saturday', 'Day_Sunday', 'Day_Thursday', 'Day_Tuesday', 'Day_Wednesday'
    ])

# Function to predict power consumption for a given date
def predict_power_consumption(date, var_model_fitted, scaler, xgb_model, rf_model, meta_model):
    future_date = pd.to_datetime(date)
    
    # Ensure the date is in the future
    if future_date <= df.index[-1]:
        raise ValueError("The date must be in the future.")
    
    # Calculate the number of steps to predict
    steps = (future_date - df.index[-1]).days
    
    # Predict future values using VAR
    var_forecast = predict_var(var_model_fitted, steps=steps)
    future_dates = pd.date_range(start=df.index[-1], periods=steps + 1, freq='D')[1:]
    
    # Create DataFrame for VAR predictions
    var_forecast_df = pd.DataFrame(var_forecast, index=future_dates, columns=['Temperature', 'Humidity', 'WindSpeed', 'GeneralDiffuseFlows', 'DiffuseFlows'])
    
    # Create static features for future dates
    future_static_df = create_static_features(future_dates)
    
    # Combine VAR predictions and static features
    future_data = pd.concat([var_forecast_df, future_static_df], axis=1)
    
    # Standardize future data using the scaler fitted on the training data
    X_future_scaled = scaler.transform(future_data)
    
    # Predict power consumption using base models
    xgb_pred_future = xgb_model.predict(X_future_scaled)
    rf_pred_future = rf_model.predict(X_future_scaled)
    
    # Stack predictions
    stacked_pred_future = np.column_stack((xgb_pred_future, rf_pred_future))
    
    # Predict power consumption using meta-model
    meta_pred_future = meta_model.predict(stacked_pred_future)
    
    # Return the prediction for the requested date
    return meta_pred_future[-1]

# Take the date as input from the user
future_date = input("Enter the future date (YYYY-MM-DD): ")

# Predict power consumption for the input date
try:
    predicted_power_consumption = predict_power_consumption(future_date, var_model_fitted, scaler, xgb_model, rf_model, meta_model)
    print(f"Predicted power consumption for {future_date}: {predicted_power_consumption}")
except ValueError as e:
    print(e)


C:\Users\91798\AppData\Local\Temp\ipykernel_16424\900167524.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Temperature'].fillna(df['Temperature'].mean(), inplace=True)
C:\Users\91798\AppData\Local\Temp\ipykernel_16424\900167524.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as 

TypeError: '<=' not supported between instances of 'Timestamp' and 'int'